- Update: 2023.07.04

# 0. 配置参数

- `source`：待查重数据源路径
- `out`：保存结果的路径

In [1]:
source = r'D:\DeskTop\Learn\3-查重自动化\Asher【模板】.xlsx'# 输入带查重文件路径

out = r'D:\DeskTop\Learn\3-查重自动化\【查重结果】发动机水管_NOW().xlsx' # 输入输出文件路径

In [2]:
import pandas as pd

In [3]:
# 读取待查重数据
fac_data = pd.read_excel(source,sheet_name='需查重数据',dtype=str).fillna('')

fac_data

## 定义清理OE函数

In [4]:
import re

In [5]:
def normalise_oe(oe):
    
    oe = str(oe).strip()
#     oe = re.sub(r'^[a-zA-Z]*:', '', oe)#去除":"及前面所有字母
#     oe = re.sub(r'[\u4e00-\u9fa5]','',oe) #去除汉字
    oe = re.sub(r'[\u4e00-\u9fa5]+[a-zA-Z]*','',oe) #去除汉字及后面所有字母
#     oe = re.sub('\.+','',oe)  #去除"."符号
    oe = re.sub('\（','(',oe) #将所有中文"（"符号替换为英文"("
    oe = re.sub('\）',')',oe) #将所有中文"）"符号替换为英文")"
    oe = re.sub('\([^)]*\)','',oe)  #删除所有"()"及其内部的字符（因为通常不是OE号）
#     oe = re.sub('\(.*\)','',oe)  #删除所有"()"及其内部的字符（因为通常不是OE号）,该方法在特殊情况下会删除OE号
    oe = re.sub('\*+', '', oe) #删除所有的"*"
    oe = re.sub('  +', ';', oe) #将两个以上的空格替换为';'
    oe = re.sub(' ', '', oe) #删除单空格
    oe = re.sub('/+', ';', oe) #将“/"替换为';'
    oe = re.sub('\n', ';', oe) #将换行符替换为';'
    oe = re.sub('[；，,+]', ';', oe) #将[；，,+]替换为';'
    oe = re.sub('\u200e', '', oe)
    oe = re.sub(';$','',oe)# 匹配末尾的';'
    oe = re.sub('\-', '', oe)# 匹配'-'

    #list1 = oe26.split(';')
    list1 = [i for i in str(oe).split(';') if i != '' and i.isalpha() == False] #去除空格
    list1 = [re.sub(r'[^0-9]+[:：\.]','',i) for i in list1]#去除":"及前面所有字母
    list1 = [re.sub(r'\.+','',i) for i in list1]
    list1 = [re.sub(r'[:：]+','',i) for i in list1]#去除":"
    list1 = [i for i in list1 if i !=''and i.isalpha() == False] #去除空格
    list1 = [re.sub(r'\r','',i) for i in list1]
    
    list1 = [oe.upper() for oe in list1] # 将OE中的字母全部转换为大写
    
    list2 = list(set(list1))
    list2.sort(key = list1.index)# 去重后保持OE号原来的顺序
    
    oe=';'.join(list2)
    
# #     删除OE号中混杂的年份如'1999';'2018'
    
#     list3 = [i for i in str(oe).split(';') if len(i) > 3]
    
#     list3 = [i for i in list3 if not (len(i) == 4 and i[0] in ['1', '2'])]

    
#     oe=';'.join(list3)
    
    return oe.upper()

In [6]:
fac_data['清理后OE'] = [normalise_oe(fac_data.loc[i, 'OE']) for i in range(len(fac_data))]

fac_data

In [7]:
# 剔除无OE的数据
clean_fac_data = fac_data[fac_data['清理后OE']!='']

clean_fac_data

In [8]:
new_fac_data = clean_fac_data[['辨识号','清理后OE']].copy()
new_fac_data['清理后OE'] = new_fac_data['清理后OE'].str.split(';')
new_fac_data = new_fac_data.explode('清理后OE')

new_fac_data

## 和其他表查重

In [9]:
# 读取系统数据
sku_data = pd.read_excel(source,sheet_name='PTSKU',dtype=str).fillna('')

sku_data

In [10]:
sku_data['清理后OE'] = [normalise_oe(sku_data.loc[i, 'OE']) for i in range(len(sku_data))]

sku_data

In [11]:
clean_sku_data = sku_data[sku_data['清理后OE']!='']

clean_sku_data

In [12]:
new_sku_data = clean_sku_data[['SKU','清理后OE']].copy()
new_sku_data['清理后OE'] = new_sku_data['清理后OE'].str.split(';')
new_sku_data = new_sku_data.explode('清理后OE')

new_sku_data

In [13]:
merge_data = pd.merge(new_fac_data,new_sku_data,on='清理后OE',how='left').fillna('')

merge_data

In [14]:
repeat_data = merge_data[merge_data['SKU']!='']

repeat_data

In [15]:
sku_repeat_data = repeat_data.groupby(['辨识号'],as_index=False,sort=False).agg({'清理后OE':';'.join,'SKU':';'.join})

sku_repeat_data

In [16]:
sku_repeat_data.rename(columns={'清理后OE':'系统重复OE','SKU':'重复PTSKU'},inplace=True)

sku_repeat_data

In [17]:
sku_repeat_data['系统重复OE'] = sku_repeat_data['系统重复OE'].str.split(';').apply(lambda x: sorted(list(set(x)), key=x.index))
sku_repeat_data['重复PTSKU'] = sku_repeat_data['重复PTSKU'].str.split(';').apply(lambda x: sorted(list((set(x))), key=x.index))

In [18]:
sku_repeat_data

In [19]:
merge_data

In [20]:
sku_repeat_data['系统重复OE'] = sku_repeat_data['系统重复OE'].apply(lambda x: ';'.join(x))
sku_repeat_data['重复PTSKU'] = sku_repeat_data['重复PTSKU'].apply(lambda x: ';'.join(x))

In [21]:
sku_repeat_data

In [22]:
result = pd.merge(fac_data,sku_repeat_data,on='辨识号',how='left').fillna('')

result

In [23]:
result['是否已开发']=''

In [24]:
result

In [25]:
for i in range(len(result)):
    if result.loc[i,'清理后OE'] != '':
        if result.loc[i,'重复PTSKU']!='':
            result.loc[i,'是否已开发'] = '是'
        else:
            result.loc[i,'是否已开发'] = '否'
    else:
        result.loc[i,'是否已开发'] = '无OE号，待进一步确认'

In [26]:
result

In [27]:
result[result['是否已开发']=='无OE号，待进一步确认']

In [28]:
import datetime

In [29]:
result.to_excel(out.replace('NOW()', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")), index=False)